# Malware triage with MISP

## Introduction

- UUID: **68b42f4d-8d5e-46c4-97f8-b87b9df210a3**
- Started from [issue 2](https://github.com/MISP/misp-playbooks/issues/2)
- State: **Published**
- Purpose: A playbook to provide an analyst sufficient information to do basic malware triage on one or more samples.
    - This playbook creates a **MISP event** for malware triage. 
    - Samples are **attached** to a MISP event (with file object relations).
    - VirusTotal and MalwareBazaar are used to get the **detection rate**, **threat classification** and **sandbox** information.
    - Hashlookup is used to check for **known hashes**.
    - PEfile analysis is done for **imports** and **exports**.
    - The results are stored in **MISP reports** and as MISP objects where relevant.
    - Correlations with MISP events or data feeds are added to a summary.
    - The sample is shared with a local instance of **MWDBcore**.
    - The summary is then sent to Mattermost.
- Tags: [ "malware", "triage", "incidentresponse", "ir", "dfir" ]
- External resources: **Mattermost
- Target audience: **SOC**, **CSIRT**, **CTI**
- Notes:
    - Samples are stored in a "malwarezoo", defined in the variable `malwarezoo`.
    - All samples that do not have an extension "do-not-run" (set in `defangsuffix`) are considered as unprocessed.
    - The playbook adds the defangsuffix when it processes the samples.
    - For VirusTotal and MalwareBazaar, both the MISP modules as well as direct queries to VT/MB are used.

# Playbook

- **Playbook title**
- - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Verify MISP modules
    - PR:3 Load helper functions
    - PR:4 Set helper variables
    - P5:5 Load and defang samples
    - PR:6 MISP event details
    - PR:7 Setup MISP event link
- **Investigate**
    - IN:1 File analysis
    - IN:2 VirusTotal results
    - IN:3 Hahslookup
    - IN:4 MalwareBazaar
    - IN:5 PE imports and exports
    - IN:6 MISP report for investigation
- **Correlation**
    - CR:1 Correlation with MISP events
    - CR:2 Correlation with MISP feeds
- **Share the sample**
    - SA:1 Store in MWDB
- **Summary**
    - EN:1 MISP indicators
    - EN:2 Create the summary of the playbook
    - EN:3 Print the input for malware triage
    - EN:4 Send a summary to Mattermost
    - EN:5 End of the playbook
    - External references
    - Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Indicate if PyMISP should attempt to verify the certificate or ignore errors
```

In [ ]:
# Initialise Python environment
import urllib3
import sys
import json
from pyfaup.faup import Faup
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
import uuid
#from uuid import uuid4
from pymisp import *
from pymisp.tools import GenericObjectGenerator
from pymisp.tools import make_binary_objects
import logging

import os
import time
import pefile
from datetime import datetime
from mwdblib import MWDB

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

# Create headers for specific cases
misp_headers = {"Authorization": misp_key,  "Content-Type": "application/json", "Accept": "application/json"}

## PR:2 Verify MISP modules

This playbook uses the MISP modules to obtain additional correlation or enrichment information. [MISP modules](https://github.com/MISP/misp-modules) are autonomous modules that can be used to extend MISP for new services such as expansion, import and export. The modules are written in Python 3 following a simple API interface. The objective is to ease the extensions of MISP functionalities without modifying core components. The API is available via a simple REST API which is independent from MISP installation or configuration.

In the next cell we check if we have access to the **MISP module** server and if the required modules are enabled.

In [ ]:
# Where can we find the local MISP Module server? You can leave this to the default setting in most cases.
misp_modules_url = "http://127.0.0.1:6666"

# How long do we wait between queries when using the MISP modules (API rate limiting of external service such as VirusTotal)
misp_modules_wait = 3

# Initiliasation
misp_modules = {}
misp_modules_headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
misp_modules_in_use = ["hashlookup", "virustotal_public", "malwarebazaar"]
# Code block to query the MISP module server and check if our modules are enabled
res = requests.get("{}/modules".format(misp_modules_url), headers=misp_modules_headers)
for module in res.json():
    for module_requested in misp_modules_in_use:
        if module.get("name", False) == module_requested:
            misp_modules[module_requested] = {"enabled": True, "input": module.get("mispattributes").get("input")}
            print("Found the \033[92m{}\033[90m MISP module (Accepted input: {}).".format(module_requested, misp_modules[module_requested]["input"]))
print("\n")

## PR:3 Load helper functions

The next cell contains **helper functions** that are used in this playbook. 

Instead of distributing helper functions as separate Python files this playbook includes all the required code as one code cell. This makes portability of playbooks between instances easier. The downside is that functions defined in this playbook need to be defined again in other playbooks, which is not optimal for code re-use. For this iteration of playbooks it is chosen to include the code in the playbook (more portability), but you can easily create one "helper" file that contains all the helper code and then import that file in each playbook (for example by adding to the previous cell `from helpers import *`). Note that the graphical workflow image is included as an external image. A missing image would not influence the further progress of the playbook.

In [ ]:
def pb_get_misp_tags(tags=[], local_tags=[]):
    '''
    Get a list of MISP tags based on a Python list

    :param misp: MISP object
    :param object_template: which object template to return
    '''
    misp_tags = []
    for el in tags:
        t = MISPTag()
        t.name = el
        t.local = False
        misp_tags.append(t)

    for el in local_tags:
        t = MISPTag()
        t.name = el
        t.local = True
        misp_tags.append(t)
    return misp_tags


def calculate_risk_score(ratio):
    '''
    Calculate risk score and return the score and a tag
    '''
    detection_ratio_perc = 0
    score = 0
    base = 0
    tag = ""
    if len(ratio) > 0:
        for detection_ratio in ratio:
            score += int(detection_ratio.split("/")[0])
            base += int(detection_ratio.split("/")[1])
        if base > 0:
            detection_ratio_perc = round((score / base) * 100, 2)
            if detection_ratio_perc > 0:
                if detection_ratio_perc >= 75:
                    tag = "misp:threat-level=\"high-risk\""
                elif detection_ratio_perc >= 50:
                    tag = "misp:threat-level=\"medium-risk\""
                #elif detection_ratio_perc >= 25:
                else:
                    tag = "misp:threat-level=\"low-risk\""
            else:
                tag = "misp:threat-level=\"no-risk\""
    return detection_ratio_perc, tag

## PR:4 Set helper variables

This cell contains **helper variables** that are used in this playbook. Their usage is explained in the next steps of the playbook.

- `samples` : a dictionary of the objects that are created when the playbook progresses
- `malwarezoo` : the location where the samples are uploaded
- `defangsuffix` : the suffix to add when defanging the samples

In [ ]:
# Dictionary to store playbook results
samples = {}

# Location to upload samples
malwarezoo = "/data/notebook-demo/playbook/notebooks/evidence"

# Suffix to add when defanging the samples
defangsuffix = ".do-not-run"

# Helper variables
misp_event = False
summary = ""

## PR:5 Load and defang samples

This section checks if there are unprocessed samples in `malwarezoo` and **loads these samples** in the dictionary `samples`. It will also defang the sample to prevent accidential execution. A sample with the `defangsuffix` suffix is considered as processed.

In [ ]:
# Check if the malware samples directory exists and misp modules are enabled
malwarezoo_errors = False

malwarezoo_samples = [file for file in os.listdir(malwarezoo) if os.path.isfile(os.path.join(malwarezoo, file))]
if len(malwarezoo_samples) == 0:
    print("There are no files in \033[91m{}\033[90m.".format(malwarezoo))
    print("\033[91mUnable to proceed!\033[90m")
else:
    # Defang the samples before we continue
    print("Defang samples")
    for sample in malwarezoo_samples:
        if defangsuffix not in sample:
            sample_path = "{}/{}{}".format(malwarezoo, sample, defangsuffix)
            sample_orig_path = "{}/{}".format(malwarezoo, sample)
            os.system("mv {} {}".format(sample_orig_path, sample_path))
            samples[sample] = {"path": sample_path, "orig_path": sample_orig_path}
            samples[sample]["MISP"] = []
            samples[sample]["Feeds"] = []
            samples[sample]["detection_ratio"] = []
            samples[sample]["detection_ratio_indirect"] = []
            samples[sample]["vt"] = {}
            samples[sample]["malwarebazaar"] = {}
            samples[sample]["hashlookup"] = {}
            samples[sample]["pe"] = {}
            print(" Renamed {} to \033[92m{}\033[90m".format(sample, sample_path))
        else:
            print(" Skip \033[91m{}\033[90m, already processed (defanged).".format(sample))
    if len(samples) > 0:
        print("\n\033[92mContinue\033[90m the playbook with \033[92m{}\033[90m samples.\n".format(len(samples)))
    else:
        print("All samples in the malwarezoo ({}) are \033[91malready processed\033[90m (defanged).\n".format(malwarezoo))

## PR:6 MISP event details

### Event title

In this playbook we create a **new** MISP event with title **Malware triage for _malwarelist_**. You get the chance to override this default title but remember that it is good practice to choose a self-explanatory **event title**. This event title is shown in the MISP event index and should provide you the necessary information what the event is about. You should avoid using generic event titles. Read the [Best Practices in Threat Intelligence](https://www.misp-project.org/best-practices-in-threat-intelligence.html) for further guidance.

### Contexualisation

This playbook adds event contexualisation via the **tags** that are defined in `event_additional_global_tags` (for *global* tags) and `event_additional_local_tags` (for *local* tags). As a reminder, whereas *global* tags remain attached to the events that you share with your community, the *local* tags are not shared outside your organisation. It's also a good idea to primarily use tags that are part of a [taxonomy](https://github.com/MISP/misp-taxonomies), this allows you to make the contexualisation more portable accross multiple MISP instances.

In this playbook the list of tags is build via one of the helper functions `pb_get_misp_tags`. This function takes two arguments, first a list of tags to convert as *global* tags, and secondly a list of tags to convert as *local* tags. It then returns a Python list of MISPTag objects.

### Traffic Light Protocol

The default **TLP** for this event is **<span style='color:#FFBF00'>tlp:amber</span>**. The Traffic Light Protocol (TLP) facilitates sharing of potentially sensitive information and allows for more effective collaboration. TLP is a set of four standard labels to indicate the sharing boundaries to be applied by the recipients. TLP is always set by the creator of information. You can find more information at [FIRST](https://www.first.org/tlp/). You can specify the TLP via `event_tlp`.

### MISP galaxies

This playbook can also add MISP galaxies to the event with the variable `event_galaxies`. You can also leave the list empty if you do not want to add galaxies in this stage of the investigation.

### MISP distribution, threat level and analysis level

Optionally you can specifiy a MISP **distribution** (with `event_distribution`), **threat level** (with `event_threat_level_id`) or **analysis state** (with `event_analysis`). The event **date** is set to today via `event_date`.

If you cannot remember the options for distribution, threat level or the analysis state then use the next cell to guide you. This cell is set as **raw**. If you **change its type to code** and execute the cell you get an overview of the options available for creating a MISP event.

## PR:7 Setup MISP event link

By default the playbook will generate a **title** with a prefix and the malware you want to investigate. You can override this event title with the variable `event_title`. If you leave this value empty the playbook will generate the MISP event title for you.

In [ ]:
# Provide the event title for a new event. Leave blank for the playbook to auto generate one
event_title = ""

# Prefix for auto generate event title
event_title_default_prefix = "Malware triage"

# Optionally, you can change TLP, add additional event (local and global) tags, threatlevel, analysis state or distribution level
event_tlp = "tlp:amber"

# Event context
event_additional_global_tags = []                                 # This needs to be a Python list
event_additional_local_tags = ["workflow:state=\"incomplete\""]   # This needs to be a Python list

# Event galaxies
event_galaxies = [ "misp-galaxy:mitre-attack-pattern=\"Malware - T1587.001\"", "misp-galaxy:mitre-attack-pattern=\"Malware - T1588.001\""]

# Additional MISP event settings
event_threat_level_id = ThreatLevel.low
event_analysis = Analysis.ongoing
event_distribution = Distribution.your_organisation_only
event_date = date.today()

### Create MISP event

The next code cell will **create the MISP event** and store the references to the newly created event in the variable `misp_event`. This variable is used further when the playbook progresses.

In [ ]:
# Code block to create the event or add data to an existing event
event_title = event_title.strip()

if not(len(event_title) > 0):
    query_sample = ""
    for key, sample in samples.items():
        query_sample = "{} {}".format(key, query_sample)
    event_title = "{} for {}".format(event_title_default_prefix, query_sample)

# Construct the event tags
event_additional_global_tags.append(event_tlp)
if len(event_galaxies) > 0:
    event_additional_global_tags.append(event_galaxies)
event_tags = pb_get_misp_tags(event_additional_global_tags, event_additional_local_tags)

# Create the PyMISP object for an event
event = MISPEvent()
event.info = event_title
event.distribution = event_distribution
event.threat_level_id = event_threat_level_id
event.analysis = event_analysis
event.set_date(event_date)

# Create the MISP event on the server side
misp_event = misp.add_event(event, pythonify=True)
print("Continue the playbook with the new \033[92mcreated\033[90m MISP event ID {} with title \033[92m{}\033[90m and UUID {}".format(misp_event.id, misp_event.info, misp_event.uuid))
for tag in event_tags:
    if len(tag) > 0:
        misp.tag(misp_event.uuid, tag, local=tag.local)
        print("\033[92mAdded\033[90m event tag {}".format(tag))
print("\n")

# Investigate

This section starts the investigation for the malware triage.

## IN:1 File analysis

The first step **uploads the samples** to the MISP event and uses the **advanced extraction** features to identify useful elements. This creates a set of related objects in the event The results are added to the `samples` dictionary.

The section temporarily sets the logging level of MISP to *error* so that it does not stop execution when errors occur when analysing specific sections of the samples.

In [ ]:
# Avoid getting PyMISP errors. We reset it to the original state afterwards
current_logging = logging.getLogger("pymisp").getEffectiveLevel()
logging.getLogger("pymisp").setLevel(logging.ERROR)

# Loop through the samples
print("Processing samples and attaching them to an event")
for key, sample in samples.items():
    attachment = sample.get("path")
    try:
        print(" Working on %s" % attachment)
        fo, peo, seos = make_binary_objects(attachment)
    except Exception:
        continue

    if seos:
        print("  PESectionObject")
        for s in seos:
            r = misp.add_object(misp_event.uuid, s)

    if peo:
        print("  PEObject")
        if hasattr(peo, 'certificates') and hasattr(peo, 'signers'):
            for c in peo.certificates:
                misp.add_object(misp_event.uuid, c, pythonify=True)
            for s in peo.signers:
                misp.add_object(misp_event.uuid, s, pythonify=True)
            del peo.certificates
            del peo.signers
        del peo.sections
        r = misp.add_object(misp_event.uuid, peo, pythonify=True)
        for ref in peo.ObjectReference:
            r = misp.add_object_reference(ref)

    if fo:
        print("  FileObject")
        response = misp.add_object(misp_event.uuid, fo, pythonify=True)
        for ref in fo.ObjectReference:
            r = misp.add_object_reference(ref)
        samples[key]["fileobject"] = response
        samples[key]["fileobject_uuid"] = response.uuid
        print("   Added file object with UUID \033[92m{}\033[90m".format(response.uuid))
        for attribute in response.attributes:
            if attribute.object_relation == "sha256":
                samples[key]["sha256"] = attribute.value
                samples[key]["sha256_uuid"] = attribute.uuid
            if attribute.object_relation == "md5":
                samples[key]["md5"] = attribute.value
            if attribute.object_relation == "malware-sample":
                samples[key]["malware-sample_uuid"] = attribute.uuid
            if attribute.object_relation == "entropy":
                samples[key]["entropy"] = attribute.value
            if attribute.object_relation == "mimetype":
                samples[key]["mimetype"] = attribute.value
        print("    MD5: \033[92m{}\033[90m\n    SHA256: \033[92m{}\033[90m".format(samples[key]["md5"], samples[key]["sha256"]))

logging.getLogger("pymisp").setLevel(current_logging)
print("Finished processing samples.")

## IN:2 VirusTotal results

Next we use [VirusTotal](https://www.virustotal.com/gui/). First we use the MISP modules to query VirusTotal. The advantage of using the MISP modules is that it returns MISP attributes and objects that can then be easily added to the event.

After using the modules, we query VirusTotal directly (without using the module) to get the **threat classification**, **sandbox** results and the **Sigma** rules that triggered detection.

Note that the sample is **not** submitted to VirusTotal for analysis.

In [ ]:
# Loop through the samples

# Use the modules to create MISP objects
# Then do a second query to get the file details

scan_count = 0
module_name = "virustotal_public"
headers = {
    "accept": "application/json",
    "x-apikey": virustotal_apikey
}

print("Verifying samples with {}".format(module_name))
for key, sample in samples.items():
    attribute_type = "sha256"
    value = sample.get("sha256")
    module_comment = "From {} for {}".format(module_name, key)
    data = {
        "attribute": {
            "type": f"{attribute_type}",
            "uuid": str(uuid.uuid4()),
            "value": f"{value}",
        },
        "module": module_name,
        "config": {"apikey": virustotal_apikey}
    }
    print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]

        for misp_attribute in result_json.get("Attribute", []):
            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
            if not "errors" in created_attribute:
                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
            else:
                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
        for misp_object in result_json.get("Object", []):
            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
            if len(misp_object["Attribute"]) > 0:
                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                if not "errors" in created_object:
                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                    if misp_object["name"] == "virustotal-report":
                        direct_vt_report = False
                        detection_ratio = False
                        for attribute in misp_object.get("Attribute", []):
                            if attribute.get("object_relation") == "permalink" and value in attribute.get("value"):
                                direct_vt_report = True
                            if attribute.get("object_relation") == "detection-ratio":
                                detection_ratio = attribute.get("value")
                        if detection_ratio:
                            if direct_vt_report:
                                samples[key]["detection_ratio"].append(detection_ratio)
                                print(" Got detection ratio \033[92m{}\033[90m ".format(detection_ratio))
                            else:
                                samples[key]["detection_ratio_indirect"].append(detection_ratio)
                                print(" Got indirect detection ratio \033[92m{}\033[90m ".format(detection_ratio))
                else:
                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        risk1, tag1 = calculate_risk_score(sample.get("detection_ratio"))
        risk2, tag2 = calculate_risk_score(sample.get("detection_ratio_indirect"))
        if len(tag1) > 0:
            misp.tag(sample.get("sha256_uuid"), tag1)
            misp.tag(sample.get("malware-sample_uuid"), tag1)

        risk_rating = "vt-detection-ratio={}\nvt-detection-ratio-indirect={}".format(risk1, risk2)
        samples[key]["detection_ratio_perc"] = risk1
        samples[key]["detection_ratio_indirect_perc"] = risk2
        samples[key]["risk_rating"] = risk_rating
        sample.get("fileobject").add_attribute("text", risk_rating)
        misp.update_object(sample.get("fileobject"))

        #################################################################

        print(" Query VirusTotal directly to get file details")
        vt_url = "https://www.virustotal.com/api/v3/files/{}".format(value)
        result = requests.get(vt_url, headers=headers)
        if "data" in result.json() and len(result.json()["data"]) > 0:
            result_json = result.json()["data"]["attributes"]
            samples[key]["vt"]["data"] = result_json
            samples[key]["vt"]["popular_threat_classification"] = result_json.get("popular_threat_classification", [])
            samples[key]["vt"]["sigma_analysis_results"] = result_json.get("sigma_analysis_results", [])
            samples[key]["vt"]["sandbox_verdicts"] = result_json.get("sandbox_verdicts", [])
            samples[key]["vt"]["sandboxes"] = []
            samples[key]["vt"]["markdown"] = ""

            if len(samples[key]["vt"]["popular_threat_classification"]) > 0:
                samples[key]["vt"]["markdown"] = "#### Popular threat classifications\n"
                for el in samples[key]["vt"]["popular_threat_classification"]:
                    if el == "suggested_threat_label":
                        samples[key]["vt"]["suggested_threat_label"] = samples[key]["vt"]["popular_threat_classification"][el]
                    samples[key]["vt"]["markdown"] = "{}\n- {}: {}".format(samples[key]["vt"]["markdown"], el, samples[key]["vt"]["popular_threat_classification"][el])

            if len(samples[key]["vt"]["sigma_analysis_results"]) > 0:
                samples[key]["vt"]["markdown"] = "{}\n\n#### Sigma analysis results\n".format(samples[key]["vt"]["markdown"])
                for el in samples[key]["vt"]["sigma_analysis_results"]:
                    samples[key]["vt"]["markdown"] = "{}\n - {}".format(samples[key]["vt"]["markdown"], el["rule_title"])

            if len(samples[key]["vt"]["sandbox_verdicts"]) > 0:
                samples[key]["vt"]["markdown"] = "{}\n\n#### Sandbox verdicts\n".format(samples[key]["vt"]["markdown"])
                for el in samples[key]["vt"]["sandbox_verdicts"]:
                    samples[key]["vt"]["sandboxes"].append(el)
                    samples[key]["vt"]["markdown"] = "{}\n - **{}**".format(samples[key]["vt"]["markdown"], el)
                    for el_sandbox in samples[key]["vt"]["sandbox_verdicts"][el]:
                        samples[key]["vt"]["markdown"] = "{}\n   - {}: {}".format(samples[key]["vt"]["markdown"], el_sandbox, samples[key]["vt"]["sandbox_verdicts"][el][el_sandbox])
        print(" Finished query VirusTotal directly")
    else:
        print("No results for \033[91m{}\033[90m.".format(value))
    scan_count += 1
    if scan_count > 5:
        print("Sleeping for {} seconds".format(misp_modules_wait))
        time.sleep(misp_modules_wait)
        scan_count = 0
print("Finished {}".format(module_name))

## IN:3 Hashlookup

We use [CIRCL Hashlookup](https://www.circl.lu/services/hashlookup/) to identify if any of the samples correspond with **known hashes**.

In [ ]:
# Loop through the samples

module_name = "hashlookup"
print("Lookup the file in {}".format(module_name))
for key, sample in samples.items():
    attribute_type = "sha256"
    value = sample.get("sha256")
    module_comment = "From {} for {}".format(module_name, key)
    data = {
        "attribute": {
            "type": f"{attribute_type}",
            "uuid": str(uuid.uuid4()),
            "value": f"{value}",
        },
        "module": module_name,
        "config": {"custom_API": False}
    }
    print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]
        hashlookup_hit = False
        hashlookuptext = ""
        for misp_attribute in result_json.get("Attribute", []):
            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
            if not "errors" in created_attribute:
                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
            else:
                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
        for misp_object in result_json.get("Object", []):
            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
            if len(misp_object["Attribute"]) > 0:
                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                if not "errors" in created_object:

                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                    for attribute in misp_object.get("Attribute", []):
                        if attribute["object_relation"] not in ["MD5", "SHA-1", "SHA-256", "SSDEEP"]:
                            hashlookuptext = "{}: {}\n{}".format(attribute["object_relation"], attribute["value"], hashlookuptext)
                        if attribute.get("object_relation") == "KnownMalicious":
                            hashlookup_hit = True
                else:
                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        if hashlookup_hit:
            misp.tag(sample.get("sha256_uuid"), "misp-workflow:analysis=\"known-file-hash\"")
            misp.tag(sample.get("malware-sample_uuid"), "misp-workflow:analysis=\"known-file-hash\"")

        sample.get("fileobject").add_attribute("text", hashlookuptext)
        samples[key]["hashlookup"]["data"] = result_json
        samples[key]["hashlookup"]["markdown"] = hashlookuptext
        misp.update_object(sample.get("fileobject"))
    else:
        print("No results for \033[91m{}\033[90m.".format(value))
print("Finished {}".format(module_name))

## IN:4 MalwareBazaar

Now we query [MalwareBazaar](https://bazaar.abuse.ch/). Similar as with VirusTotal, we use the MISP module to get useful attributes and objects. Afterwards we query MalwareBazaar directly to get more details such as vendor intel and tags.

In [ ]:
# Loop through the samples

module_name = "malwarebazaar"
print("Lookup the file in {}".format(module_name))
for key, sample in samples.items():
    attribute_type = "sha256"
    value = sample.get("sha256")
    module_comment = "From {} for {}".format(module_name, key)
    data = {
        "attribute": {
            "type": f"{attribute_type}",
            "uuid": str(uuid.uuid4()),
            "value": f"{value}",
        },
        "module": module_name,
    }
    print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]
        for misp_attribute in result_json.get("Attribute", []):
            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
            if not "errors" in created_attribute:
                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
            else:
                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
        for misp_object in result_json.get("Object", []):
            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
            if len(misp_object["Attribute"]) > 0:
                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                if not "errors" in created_object:
                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                else:
                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        #################################################################

        print(" Query MalwareBazaar directly to get file details")
        url = "https://mb-api.abuse.ch/api/v1/"
        result = requests.post(url, data={"query": "get_info", "hash": value})
        if "data" in result.json() and len(result.json()["data"]) > 0:
            result_json = result.json()["data"][0]
            samples[key]["malwarebazaar"]["markdown"] = ""
            samples[key]["malwarebazaar"]["data"] = result_json
            samples[key]["malwarebazaar"]["tags"] = result_json.get("tags", "")
            samples[key]["malwarebazaar"]["signature"] = result_json.get("signature", "")

            if len(samples[key]["malwarebazaar"]["tags"]) > 0:
                samples[key]["malwarebazaar"]["markdown"] = "{}\n**Tags**\n".format(samples[key]["malwarebazaar"]["markdown"])
                for tag in samples[key]["malwarebazaar"]["tags"]:
                    samples[key]["malwarebazaar"]["markdown"] = "{}\n - {}\n".format(samples[key]["malwarebazaar"]["markdown"], tag)

            if len(result_json.get("vendor_intel", [])) > 0:
                if len(result_json.get("vendor_intel", []).get("Triage", [])) > 0:
                    samples[key]["malwarebazaar"]["markdown"] = "{}\n**Triage tags and signatures**\n".format(samples[key]["malwarebazaar"]["markdown"])
                    for tag in result_json.get("vendor_intel", []).get("Triage", []).get("tags", []):
                        samples[key]["malwarebazaar"]["markdown"] = "{}\n - {}\n".format(samples[key]["malwarebazaar"]["markdown"], tag)
                    for signature in result_json.get("vendor_intel", []).get("Triage", []).get("signatures", []):
                        samples[key]["malwarebazaar"]["markdown"] = "{}\n - {}\n".format(samples[key]["malwarebazaar"]["markdown"], signature["signature"])
        print(" Finished query MalwareBazaar directly")
    else:
        print("No results for \033[91m{}\033[90m.".format(value))
print("Finished {}".format(module_name))

## IN:5 PE imports and exports

Next we use `pefile` to analyse the **imports and exports**. The results are added as a **MISP report**.

In [ ]:
summary_pe = "## PEFile summary\n\n"
print("Using pefile")
for key, sample in samples.items():
    summary_pe += "## {}\n\n".format(sample["path"])

    try:
        pe = pefile.PE(sample["path"])
        samples[key]["pe"]["imports"] = "### Imports\n\n"
        samples[key]["pe"]["exports"] = "### Exports\n\n"
        pe.parse_data_directories()

        print(" Reading imports for {}".format(sample["path"]))
        try:
            for entry in pe.DIRECTORY_ENTRY_IMPORT:
                #print("  {}".format(entry.dll))
                samples[key]["pe"]["imports"] = "{}\n**{}**\n".format(samples[key]["pe"]["imports"], entry.dll)
                for imp in entry.imports:
                    samples[key]["pe"]["imports"] = "{}\n - {} {}\n".format(samples[key]["pe"]["imports"], hex(imp.address), imp.name)
        except:
            print("  No imports")
            samples[key]["pe"]["imports"] = "{}\n *No imports*\n".format(samples[key]["pe"]["imports"])
        summary_pe += samples[key]["pe"]["imports"]

        print(" Reading exports for {}".format(sample["path"]))
        try:
            for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
                samples[key]["pe"]["exports"] = "{}\n - {} {} {}\n".format(samples[key]["pe"]["exports"], hex(pe.OPTIONAL_HEADER.ImageBase + exp.address), exp.name, exp.ordinal)
        except:
            print("  No exports")
            samples[key]["pe"]["exports"] = "{}\n *No exports*\n".format(samples[key]["pe"]["exports"])
        summary_pe += samples[key]["pe"]["imports"]
    except:
        print(" \033[91mPEFormatError\033[90m for {}\n".format(sample["path"]))
        summary_pe += "\n*PEFormatError*\n"

event_title = "PEFile analysis"
print("\nCreating MISP report \033[92m{}\033[90m".format(event_title))
chunk_size = 61500
for i in range(0, len(summary_pe), chunk_size):
    chunk = summary_pe[i:i + chunk_size]
    event_report = MISPEventReport()
    event_title_edit = event_title
    if i > 0:
        event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
    event_report.name = event_title_edit
    event_report.content = chunk
    result = misp.add_event_report(misp_event.id, event_report)
    if "EventReport" in result:
        print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
    else:
        print("Failed to create report for \033[91m{}\033[90m.".format(event_title))

## IN:6 MISP report for investigation

Apart from the report on PE data, we also create a report on the previous investigation steps.

In [ ]:
summary_iv = "## Investigation report\n\n"
current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")
for key, sample in samples.items():
    summary_iv += "## Analysis for {}\n".format(key)
    summary_iv += "- Date: **{}**\n".format(formatted_date)
    summary_iv += "- MISP event: **{}** ({})\n".format(misp_event.info, misp_event.id)
    summary_iv += "### File details\n"
    summary_iv += " - Path: {}\n".format(sample["path"])
    summary_iv += " - MD5: {}\n".format(sample["md5"])
    summary_iv += " - SHA256: {}\n".format(sample["sha256"])
    summary_iv += " - Entropy: {}\n".format(sample["entropy"])
    summary_iv += " - Mime-type: {}\n".format(sample["mimetype"])
    summary_iv += "### Risk\n"
    if sample.get("detection_ratio", False):
        summary_iv += " - Detection ratio: **{}**\n".format(sample["detection_ratio"])
        summary_iv += " - Detection ratio percentage: **{}**\n".format(sample["detection_ratio_perc"])
        summary_iv += " - Detection ratio indirect: {}\n".format(sample["detection_ratio_indirect"])
        summary_iv += " - Detection ratio indirect percentage: {}\n".format(sample["detection_ratio_indirect_perc"])
    else:
        summary_iv += "*No detection ratio found*\n\n"
    if sample.get("risk_rating", False):
        summary_iv += " - Risk rating: **{}**\n".format(sample["risk_rating"])
    summary_iv += "### VirusTotal\n"
    if sample["vt"].get("markdown"):
        summary_iv += sample["vt"]["markdown"]
    else:
        summary_iv += "*No results*\n"
    summary_iv += "\n\n"
    summary_iv += "### Hashlookup\n"
    if sample["hashlookup"].get("markdown"):
        summary_iv += sample["hashlookup"]["markdown"]
    else:
        summary_iv += "*No results*\n"
    summary_iv += "\n\n"
    summary_iv += "### MalwareBazaar\n"
    if sample["malwarebazaar"].get("markdown"):
        summary_iv += sample["malwarebazaar"]["markdown"]
    else:
        summary_iv += "*No results*\n"
    summary_iv += "\n\n"

event_title = "Malware analysis"
print("Creating MISP report \033[92m{}\033[90m".format(event_title))
chunk_size = 61500
for i in range(0, len(summary_iv), chunk_size):
    chunk = summary_iv[i:i + chunk_size]
    event_report = MISPEventReport()
    event_title_edit = event_title
    if i > 0:
        event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
    event_report.name = event_title_edit
    event_report.content = chunk
    result = misp.add_event_report(misp_event.id, event_report)
    if "EventReport" in result:
        print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
    else:
        print("Failed to create report for \033[91m{}\033[90m.".format(event_title))

# Correlation

## CR:1 Correlation with MISP events

This cell searches the **MISP server** for events that have a match with the analysed samples.

Only **published** events (`correlation_published`) and attributes that have the **to_ids** flag (`correlation_to_ids`) set are take into account. There is a default limit of **1000 hits** (`correlation_limit`) and you can limit the search with tags (`correlation_match_tags`).

In [ ]:
# Only query for published MISP events
correlation_published = False

# Only consider those values that have the to_ids field set to True
correlation_to_ids = True

# Limit the returned results to 1000 attributes
correlation_limit = 1000

# Only return results corresponding with these tags
correlation_match_tags = ["tlp:amber", "tlp:white"]

In [ ]:
print("Search for correlating MISP events")
# Code block to query MISP and find the correlations
for key, sample in samples.items():
    value = [sample.get("sha256"), sample.get("md5")]
    search_match = misp.search("attributes", to_ids=correlation_to_ids, value=value, tags=correlation_match_tags,
                                   published=correlation_published, limit=correlation_limit, pythonify=True)
    if len(search_match) > 0:        
        for attribute in search_match:
            if attribute.Event.id != misp_event.id:   # Skip the event we just created for this playbook
                print(" Found \033[92m{}\033[90m in \033[92m{}\033[90m ({})".format(attribute.value, attribute.Event.info, attribute.Event.id ))
                entry = {"source": "MISP", "category": attribute.category, "type": attribute.type, "event_id": attribute.Event.id, "event_info": attribute.Event.info}
                samples[key]["MISP"].append(entry)
print("Finished searching for correlations")

### MISP events correlation table

The correlation results are now stored in `samples`. Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Category", "Type", "Event", "Event ID"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Event"] = "l"
table.align["Event ID"] = "l"
table._max_width = {"Event": 50}
for key, sample in samples.items():
    for match in sample["MISP"]:
        table.add_row([match["source"], key, match["category"], match["type"], match["event_info"], match["event_id"]])
print(table.get_string(sortby="Value"))
table_mispevents = table

## CR:2 Correlation with MISP feeds

This cell searches the **MISP feeds** for events that have a match with the analysed samples. The output of this cell is a table with all the matches. The output is also repeated at the end of the playbook.

Note that the correlation lookup in the MISP feeds does not return the name of the MISP event, it returns the UUID of the event as title.

In [ ]:
print("Search in MISP feeds")
misp_cache_url = "{}/feeds/searchCaches/".format(misp_url)
match = False
for key, sample in samples.items():
    # Instead of GET, use POST (https://github.com/MISP/MISP/issues/7478)
    value = sample.get("sha256")
    cache_results = requests.post(misp_cache_url, headers=misp_headers, verify=misp_verifycert, json={"value": value})
    for result in cache_results.json():
        if "Feed" in result:
            match = True
            print(" Found \033[92m{}\033[90m in \033[92m{}\033[90m".format(value, result["Feed"]["name"]))
            for match in result["Feed"]["direct_urls"]:
                entry = {"source": "Feeds", "feed_name": result["Feed"]["name"], "match_url": match["url"]}
                samples[key]["Feeds"].append(entry)

print("Finished searching in MISP feeds")
if not match:
    print("\033[93mNo correlating information found in MISP feeds.")

### MISP feed correlations table

The correlation results are now stored in `samples`. Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Feed", "URL"]
table.align["Value"] = "l"
table.align["Feed"] = "l"
table.align["Feed URL"] = "l"
table._max_width = {"Feed": 50}
for key, sample in samples.items():
    for match in sample["Feeds"]:
        table.add_row([match["source"], key, match["feed_name"], match["match_url"]])
print(table.get_string(sortby="Value"))
table_mispfeeds = table

# Share the sample

## SA:1 Store in MWDB

The Malware Repository MWDB, formerly known as Malwarecage, is a project from CERT.pl that is available as a service (via [MWDB-CERT.pl](https://mwdb.cert.pl/login) but you can also host its core component [MWDB](https://github.com/CERT-Polska/mwdb-core) on your own infrastructure. In this playbook there's the option to store the sample in your instance of MWDB. Future variants of the playbook include sharing the sample with well-known malware sandboxes.

There is a MISP module that allows to upload samples from MISP to MWDB, but in this case the playbook will interact directly with MWDB.

In [ ]:
mwdb_public = True
mwdb_metakeys = False
mwdb = MWDB(api_key=mwdb_apikey, api_url=mwdb_api_url)
mwdb_tags = ["misp", "playbook"]

summary_mwdb = "## Samples stored in MWDB\n"
print("Start sharing with MWDB")
for key, sample in samples.items():
    sample_filename = key
    with open(sample["path"], 'rb') as file:
        data = file.read()
    file_object = mwdb.upload_file(sample_filename, data, metakeys=mwdb_metakeys, public=mwdb_public)
    for tag in mwdb_tags:
        file_object.add_tag(tag)
    file_object.add_comment("Uploaded from MISP playbook for malware triage, via event {}".format(misp_event.id))
    mwdb_link = "{}{}".format(mwdb_api_url.replace("/api", "/file/"), file_object.md5)
    attribute = MISPAttribute()
    attribute.value = mwdb_link
    attribute.to_ids = False
    attribute.type = "link"
    attribute.disable_correlation = True
    attribute.comment = "MWDB link for sample {}".format(key)
    attribute_mwdb = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
    summary_mwdb += " {} in [{}]({})\n".format(key, mwdb_link, mwdb_link)
    if not "errors" in attribute_mwdb:
        misp.add_object_reference(sample["fileobject"].add_reference(attribute_mwdb.uuid, "related-to"))
    else:
        print(" Unable to add attribute with link to MWDB to MISP")
    print(" Sample {} at \033[92m{}\033[90m".format(key, mwdb_link))
summary_mwdb += "\n"
print("Finished sharing")

# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed in the playbook and can also be send to a chat channel. 

## EN:1 MISP indicators

The next section first **queries MISP for the indicators added to the MISP event** that is linked to the execution of this playbook.

The indicators are stored in the variable `indicator_table` (table format) and `indicator_raw_list` (in raw format) which is used in a later section to create the playbook summary.

In [ ]:
# Get all the indicators for our event and store this is in a table. We can also use this for the summary.
indicator_search = misp.search("attributes", uuid=misp_event.uuid, to_ids=True, pythonify=True)
indicator_raw_list = []
indicator_table = PrettyTable()
if len(indicator_search) > 0:
    indicator_table.field_names = ["Type", "Category", "Indicator", "Comment"]
    indicator_table.align["Type"] = "l"
    indicator_table.align["Category"] = "l"
    indicator_table.align["Indicator"] = "l"
    indicator_table.align["Comment"] = "l"
    indicator_table.border = True
    for indicator in indicator_search:
        if indicator.value not in indicator_raw_list:
            comment = indicator.comment
            if hasattr(indicator, 'object_relation'):
                object_ind = misp.get_object(indicator.object_id, pythonify=True)
                comment = "From '{}' object {} {}".format(object_ind.name, object_ind.comment.lower(), comment.lower())
            indicator_table.add_row([indicator.type, indicator.category, indicator.value, comment])
            indicator_raw_list.append(indicator.value)
    print("Got \033[92m{}\033[90m indicator(s) from the event \033[92m{}\033[90m ({}).\n".format(len(indicator_raw_list), misp_event.info, misp_event.id))
else:
    print("\033[93mNo indicators found in the event \033[92m{}\033[90m ({})".format(misp_event.info, misp_event.id))

### Raw list of MISP indicators

The indicators are now stored in `indicator_search` (as Python objects) and `indicator_raw_list` (in raw format, only the indicators). Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [ ]:
if len(indicator_raw_list) > 0:
    print(indicator_table.get_string(sortby="Type"))
    print("\n\nIndicator list in raw format:")
    print("---------------------------------------------------")
    for el in indicator_raw_list:
        print("{}".format(el))
    print("---------------------------------------------------")

## EN:2 Create the summary of the playbook

The next section creates a summary and stores the output in the variable `summary` in Markdown format. It also stores an intro text in the variable `intro`. These variables are later used when sending information to Mattermost or TheHive.

In [ ]:
summary = "# MISP Playbook summary\nMalware triage with MISP event: **{}** ({}/events/view/{}). ".format(misp_event.info, misp_url, misp_event.id)

summary += "\n"
summary += summary_iv
summary += "\n"
intro = summary

summary += "## Indicators\n\n"
summary += "### Indicators table\n\n"
if len(indicator_raw_list) > 0:
    indicator_table.set_style(MARKDOWN)
    summary += indicator_table.get_string(sortby="Type")
    summary += "\n\n\n" 
    summary += "### Indicators in **raw format**\n\n"
    for indicator in indicator_raw_list:
        summary += "{}\n\n".format(indicator)
    summary += "\n" 
else:
    summary += "There are no indicators"
summary += "\n\n"

summary += "## Correlations\n\n"
summary += "### MISP event matches\n\n"
table_mispevents.set_style(MARKDOWN)
summary += table_mispevents.get_string()
summary += "\n\n"

summary += "### MISP feed matches\n\n"
table_mispfeeds.set_style(MARKDOWN)
summary += table_mispfeeds.get_string()
summary += "\n\n"

summary += summary_pe

summary += "\n\n"

print("The \033[92msummary\033[90m of the playbook is available.\n")

## EN:3 Print the input for malware triage

Apart from the full summary of the investigation, the malware triage summary provides the necessary input for the analyst. This summary was previously also added as a MISP report.

In [ ]:
print(summary_iv)
# Or print with parsed markdown
# display_markdown(summary_iv, raw=True)

## EN:4 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [ ]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [ ]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

## EN:5 End of the playbook 

In [ ]:
print("\033[92m End of the playbook")


## External references <a name="extreferences"></a>

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [MWDB](https://github.com/CERT-Polska/mwdb-core)
- [MalwareBazaar](https://bazaar.abuse.ch/)
- [Hashlookup](https://www.circl.lu/services/hashlookup/)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
pyfaup
chardet
PrettyTable
ipywidgets
pefile
mwdblib
```

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```